# Comparision of Systematic and Random Scan Gibbs Sampling algorithms

We have seen in the previous two tutorials how to run the Gibbs samplers for both the Systematic scan and Random scan regimes. In this notebook we will compare the two approaches, both in terms of _execution time_ and _accuracy_.

In [1]:
# Import required libraries
import seqgibbs

import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import time

## Execution time

We run 10 times each of the samplers for 200 steps for the same bivariate normal distribution as in the tutorials:

Let $X := (X_1,X_2) \sim N(\mu, \Sigma)$ where $\mu = (\mu_1, \mu_2)$ and covariance matrix 
$
\Sigma = \begin{pmatrix}
\sigma^2_1 & \rho \\
\rho & \sigma^2_2
\end{pmatrix}
$
.



The Gibbs sampler proceeds as follows in this case with the following conditional probabilities:

(a) Sample $X_1^{(t)} \sim N (\mu_1 +\rho/\sigma_2^2(X_2^{(t−1)}−\mu_2), \sigma_1^2−\rho^2/\sigma_2^2)$

(b) Sample $X_2^{(t)} \sim N (\mu_2 +\rho/\sigma_1^2(X_1^{(t)}−\mu_1), \sigma_2^2−\rho^2/\sigma_1^2)$.

### Systematic Scan

In [3]:
# Set correlation coefficient
rho = 0.99

# Set other parameters of the target distribution
mu1, mu2, var1, var2 = (5, 5 , 2, 1)

# Create now functions that return the parameters for the unidimensional simulator
# for X1 and X2 in terms of the current position.
# The sampler has a normal distribution shape so these parameters need
# to be the mean and variance of the distribution.
def first_fun(x):
    return mu1 + rho/var2 *(x - mu2), var1 - rho**2/var2

def second_fun(x):
    return mu2 + rho/var1 *(x - mu1), var2 - rho**2/var1

# Now create the unidimensional samplers for each of the two dimensions X1 and X2
first_sampler = seqgibbs.OneDimSampler(scipy.stats.norm.rvs, first_fun)
second_sampler = seqgibbs.OneDimSampler(scipy.stats.norm.rvs, second_fun)

# Now create wrapper Systematic Scan Gibbs sampler starting at the default position (origin)
# to which we then feed the two unidimensional samplers we constructed.
sys_sampler = seqgibbs.SysGibbsAlgo(num_dim=2, initial_state=np.array([0, 0]))

sys_sampler.add_1_d_sampler(first_sampler)
sys_sampler.add_1_d_sampler(second_sampler)

sys_times = np.zeros(10)
for t, _ in enumerate(sys_times):
    start = time.perf_counter()
    sys_sampler.run(num_cycles=200)
    end = time.perf_counter()
    sys_times[t] = end - start

[0.02811474 0.02725153 0.02498692 0.02803746 0.02579652 0.02510296
 0.02565845 0.02657802 0.0218375  0.02190983]


### Random Scan

In [5]:
# Now create wrapper Random Scan Gibbs sampler starting at the default position (origin)
# and equal probabilty to update either dimension
# and to which we then feed the two unidimensional samplers we constructed.
rand_sampler = seqgibbs.RandGibbsAlgo(num_dim=2, initial_state=np.array([0, 0]))
rand_sampler = seqgibbs.RandGibbsAlgo(num_dim=2, initial_state=np.array([0, 0]))

rand_sampler.add_1_d_sampler(first_sampler)
rand_sampler.add_1_d_sampler(second_sampler)

rand_times = np.zeros(10)
for t, _ in enumerate(rand_times):
    start = time.perf_counter()
    rand_sampler.run(num_cycles=200)
    end = time.perf_counter()
    rand_times[t] = end - start

[0.0247822  0.02129883 0.02051419 0.01939708 0.0178803  0.01883589
 0.0188401  0.01945978 0.02116241 0.01833729]


### Comparison

In [20]:
import pandas as pd
indices = [str(i) for i in range(1, 11)]
indices.append('mean')

comparison_time_matrix = pd.DataFrame(
    {
    "Systematic Scan": np.append(sys_times, np.average(sys_times)),
    "Random Scan": np.append(rand_times, np.average(rand_times))
}, index=indices)

print(comparison_time_matrix)

      Systematic Scan  Random Scan
1            0.028115     0.024782
2            0.027252     0.021299
3            0.024987     0.020514
4            0.028037     0.019397
5            0.025797     0.017880
6            0.025103     0.018836
7            0.025658     0.018840
8            0.026578     0.019460
9            0.021838     0.021162
10           0.021910     0.018337
mean         0.025527     0.020051


## Accuracy